In [1]:
!pip install transformers peft bitsandbytes trl deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.2/289.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 37.9 MB/s eta 0:00:00
     ━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

In [4]:
# setup the quantization config

compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

In [6]:
# from google.colab import userdata
# userdata.get('Llama3')

In [6]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B


# Load base model with access token
base_model_name = 'meta-llama/Meta-Llama-3-8B'
access_token = "#"
llama_3 = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    token=access_token,
    quantization_config=quant_config,
    device_map={"": 0},
)

# Load tokenizer with access token
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    token=access_token,
    trust_remote_code=True,
)

# Set padding token and side
tokenizer.pad_token = tokenizer.eos_token # this model doesnt requires separate padding token
tokenizer.padding_side = "right" #padding should be added to the end (right side) of the sequences

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
import re

def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Fixed template values
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"
    validity_date = "1 DECEMBER 2024"

    # Determine discount offer and coupon code based on interest level
    discount_map = {
        "very interested": ("30%", "NEPAL30"),
        "interested": ("20%", "NEPAL20"),
        "somewhat interested": ("15%", "NEPAL15"),
        "not interested": ("15%", "NEPAL15")
    }
    discount_offers, coupon_code = discount_map.get(interest_level, ("15%", "NEPAL15"))

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")  # Default to informative if interest level is unknown

    # Structured input for the email generation
    structured_input = (
        f"Generate a personalized email in paragraph form with the following details, but do not generate information about sales, promotions , discounts, prices, discount codes or dates:\n"
        f"Product Name: {product_name}\n"
        f"Product Description: {product_description}\n"
        f"Interest Level: {interest_level}\n"
        f"User Name: {user_name}\n"
        f"Email Tone: {tone}\n"
        f"Email Content in Paragraph:\nSubject: Exclusive {product_name} Offer Just for You!\n\n\n"
        f"Dear {user_name},\n\n\n"
    )

    # Tokenize the structured input
    inputs = tokenizer(structured_input, return_tensors="pt")

    # Generate the output
    outputs = llama_3.generate(
        inputs.input_ids,
        max_length=500,  # Adjusting max length for approximately 150 words
        min_length=200,  # Ensuring minimum length for completeness
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,  # Prevent repetition
    )

    # Decode the generated text
    email = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated email content
    email_content_start = email.find("Subject:")  # Start of the generated content
    generated_email = email[email_content_start:].strip()

    # Ensure the email is approximately 200 words
    words = generated_email.split()
    if len(words) > 200:
        generated_email = " ".join(words[:200]) + "..."

    # Remove any existing sign-offs
    sign_offs = ["Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best", "money would be refunded", "company would be liable", "product can be returned after used", "simply use the discount"]
    for sign_off in sign_offs:
        if sign_off in generated_email:
            generated_email = generated_email.split(sign_off)[0].strip()

    # Guard against generated coupon codes
    forbidden_phrases = ["code", "coupon code", "discount code", "offer code", "Discount Offer", "retailed price", "retailed rate","discounted price", "discounted rate"]
    for phrase in forbidden_phrases:
        if phrase in generated_email:
            generated_email = generated_email.split(phrase)[0].strip()

    # Remove any URLs or Links
    generated_email = re.sub(r'http\S+', '', generated_email)
    generated_email = re.sub(r'www\.\S+', '', generated_email)
    generated_email = re.sub(r'<.*?>', '', generated_email)
    generated_email = re.sub(r'\d+% off', '', generated_email)

    #     # Explicitly extract and format the subject line
    # subject_match = re.search(r'Subject: (.+)', generated_email)
    # if subject_match:
    #     subject = subject_match.group(1)
    #     generated_email = generated_email.replace(f"Subject: {subject}", f"Subject: {subject}\n\n")
    # else:
    #     generated_email =  "Subject: Exclusive " + product_name + " Offer Just for You!\n\n" + generated_email


    # Append the fixed template values with dynamic coupon code and discount offer
    generated_email += (
        f"\n\nUse the coupon code '{coupon_code}' to get {discount_offers} off. Valid till {validity_date}.\n"
        f"\nBest regards,\n"
        f"{sender_name}\n"
        f"{company_name}\n"
        f"Visit us: {website_url}\n"
    )

    return generated_email




In [49]:
# Inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "interest_level": "very interested",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        interest_level=inputs["interest_level"],
        user_name=inputs["user_name"]
    )
    print(email_content)

Subject: Exclusive Beer Offer Just for You!


Dear Ms. Sriju,


You have been selected for an exclusive offer on Beer! We understand that you are a fan of this popular alcoholic drink and we are thrilled to provide you with an amazing deal on our products. Our beers are crafted with the finest ingredients and offer a range of delicious flavors to suit every taste bud. Whether you prefer a light and refreshing lager or a dark and robust stout, we have something for everyone. Our products are available at affordable prices and we offer a variety of discounts and promotions to make your purchase even more rewarding. We hope that this offer will help you enjoy your favorite drink even more. Please feel free to contact us if you have any questions or concerns. We look forward to serving you soon!

Use the coupon code 'NEPAL30' to get 30% off. Valid till 1 DECEMBER 2024.

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/



In [50]:
# Inputs
inputs_list = [
    {
        "product_name": "Beer",
        "product_description": "Beer is a popular alcoholic beverage made from fermented grains, typically barley, and flavored with hops, offering a wide range of flavors and styles from light and crisp to dark and rich.",
        "interest_level": "interested",
        "user_name": "Ms. Sriju"
    }
]

# Generate and print emails for each set of inputs
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        interest_level=inputs["interest_level"],
        user_name=inputs["user_name"]
    )
    print(email_content)


Subject: Exclusive Beer Offer Just for You! Dear Ms. Sriju, We're excited to offer you our exclusive beer offer just for you! Our team has curated a list of the top beers from around the world, carefully selected to suit your taste and preferences. We've included a variety of styles and flavors, from light lagers to dark stouts, so you can find the perfect beer for any occasion. Our list includes some of the most sought-after beers in the world. From Belgian ales to German pilsners, we've got something for everyone. We also offer a wide selection of craft beers, so if you're looking for something unique and local, we have you covered. We hope you enjoy our curated list of beers. Please feel free to reach out to us if you have any questions or feedback. We're always here to help!

Use the coupon code 'NEPAL20' to get 20% off. Valid till 1 DECEMBER 2024.

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/



In [51]:
# Inputs
inputs_list = [
    {
        "product_name": "Noise-Cancelling Headphones",
        "product_description": "Immerse yourself in your favorite music or podcasts with our premium noise-cancelling headphones. Enjoy crystal-clear audio and a comfortable fit for hours of listening pleasure.",
        "interest_level": "not interested",
        "user_name": "Mr. Rajesh"
    }
]

# Generate and print emails (unchanged)
for inputs in inputs_list:
    email_content = generate_email_auto(
        product_name=inputs["product_name"],
        product_description=inputs["product_description"],
        interest_level=inputs["interest_level"],
        user_name=inputs["user_name"]
    )
    print(email_content)


Subject: Exclusive Noise-Cancelling Headphones Offer Just for You!


Dear Mr. Rajesh,


We are excited to offer you an exclusive discount on our premium
noise-cancalling headphones. As a valued customer, we want to make sure
you have access to the best products at the most competitive prices. Our
headphones feature advanced noise-cancellation technology, allowing
you to enjoy your favorite tunes without any distractions. With a
comfortable fit and durable construction, these headphones are perfect
for long listening sessions.


To take advantage of this limited-time offer, please use the following
discount

Use the coupon code 'NEPAL15' to get 15% off. Valid till 1 DECEMBER 2024.

Best regards,
Binit Sapkota
MrBeerGod Technologies
Visit us: https://barahsinghe.com/

